## TensorFlow Recurrent Neural Networks (Complete guide with examples and code)

Click the image below to read the post online.

<a target="_blank" href="https://www.machinelearningnuggets.com/tensorflow-lstm
"><img src="https://www.machinelearningnuggets.com/ezoimgfmt/digitalpress.fra1.cdn.digitaloceanspaces.com/mhujhsj/2022/07/logho-1.png?ezimgfmt=ng:webp/ngcb1" alt="Open in ML Nuggets"></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding, SpatialDropout1D
import matplotlib.pyplot as plt
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
pip install wget

In [ ]:
import wget
wget.download("https://ml.machinelearningnuggets.com/complaints.csv")

In [ ]:
stopWords = set(stopwords.words('english'))

In [ ]:
complaints = pd.read_csv("complaints.csv")

In [ ]:
complaints = complaints[['Consumer complaint narrative','Product']]

In [ ]:
complaints.dropna(inplace=True)

In [ ]:
symbols_regex = re.compile('[/(){}\[\]\|@,;]')
bad_symbols_regex = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    text = text.replace('\d+','')
    text = text.lower()
    text = symbols_regex.sub(' ', text)
    text = bad_symbols_regex.sub('', text) 
    text = text.replace('x', '')
    return text
complaints['Consumer complaint narrative'] = complaints['Consumer complaint narrative'].apply(clean_text)


In [ ]:
complaints

In [ ]:
complaints['Product'].value_counts().sort_values(ascending=False)

In [ ]:
len(complaints['Product'].unique())

In [ ]:
complaints_text = complaints["Consumer complaint narrative"]

In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',max_tokens=5000,output_mode='int',output_sequence_length=512)
vectorize_layer.adapt(complaints_text,batch_size=None)

In [ ]:
X_train_padded =  vectorize_layer(complaints_text)
X_train_padded = X_train_padded.numpy()

In [ ]:
X_train_padded = np.reshape(X_train_padded, (np.shape(X_train_padded)[0], np.shape(X_train_padded)[1], 1 ))

In [ ]:
le = LabelEncoder()
complaints['Product'] = le.fit_transform(complaints['Product'])


In [ ]:
y = complaints['Product']
y = tf.keras.utils.to_categorical(y, 9)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_padded,y, test_size = 0.3, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
classifier = Sequential()
classifier.add(Embedding(50000, 100, input_length=X_train.shape[1]))
classifier.add(SpatialDropout1D(0.2))
classifier.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
classifier.add(Dense(9, activation='softmax'))

In [ ]:
classifier.summary()

In [ ]:
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = classifier.fit(X_train, y_train, epochs=5, batch_size=64,validation_split=0.1,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [ ]:
y_preds = classifier.predict(X_test)

In [ ]:
print(y_test.shape)

## Where to go from here
Follow us on [LinkedIn](https://www.linkedin.com/company/mlnuggets), [Twitter](https://twitter.com/ml_nuggets), [GitHub](https://github.com/mlnuggets) and subscribe to our [blog](https://www.machinelearningnuggets.com/#/portal) so that you don't miss a new issue.